### 중복 확인 및 제거

In [8]:
import pandas as pd

In [3]:
total_raw = pd.read_csv('./total_raw.csv')
total_raw.head()

,ko,en,source
0,통영 용남면 장평리 바닷가 해안도로 앞에 위치한 통영 늘푸른 펜션에 오신 것을 환영...,Welcome to Tongyeong's Evergreen Pension locat...,111
1,허브아일랜드의 불빛동화 축제는 국내 최고 겨울축제 중 하나로 꼽힌다.,Herb Island's Light Fairy Tale Festival is con...,111
2,봉비산 자락에 위치하고 있는 도심 속의 전통 사찰이다.,It is a traditional temple in the city center ...,111
3,수룡동 마을안길로 들어가면 쉽게 오를 수 있다.,It can be easily climbed by entering the insid...,111
4,"토스 측은 ""가족과 함께 송편을 나눠먹듯, 송편지원금 또한 가족, 친척, 친구, 오...","Toss explained the meaning of the event, sayin...",111


In [29]:
value_counts_raw = total_raw['source'].value_counts()
print(value_counts_raw.sort_index())

source
111      1350000
124      1344631
125      1361845
126      1602418
563       359999
71265    2700345
71266    1350162
71382     587084
Name: count, dtype: int64


In [18]:
total_koen = total_raw[['ko', 'en']]
duplicated_indices = total_koen.duplicated()
print(sum(duplicated_indices))

239975


In [22]:
unique_indices = total_koen[~duplicated_indices].index
total_df = total_raw.loc[unique_indices]

In [31]:
value_counts_drop = total_df['source'].value_counts()
print(value_counts_drop.sort_index())

source
111      1350000
124      1344631
125      1361825
126      1599924
563       358424
71265    2486058
71266    1328987
71382     586660
Name: count, dtype: int64


In [24]:
print(len(total_raw))
print(len(total_df))
print(len(total_raw) - len(total_df))

10656484
10416509
239975


In [32]:
value_counts_diff = value_counts_raw - value_counts_drop
print(value_counts_diff.sort_index())

source
111           0
124           0
125          20
126        2494
563        1575
71265    214287
71266     21175
71382       424
Name: count, dtype: int64


In [33]:
total_df.to_csv('./total.csv', index=False)

In [4]:
total_df = pd.read_csv('./total.csv')

In [44]:
print("# of data:", len(total_df))
print(total_df['source'].value_counts(normalize=True))

# of data: 10416509
source
71265    0.238665
126      0.153595
125      0.130737
111      0.129602
124      0.129087
71266    0.127585
71382    0.056320
563      0.034409
Name: proportion, dtype: float64


### Train/Eval/Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_df, eval_test_df = train_test_split(total_df, train_size=0.8, random_state=42)
eval_df, test_df = train_test_split(eval_test_df, train_size=0.5, random_state=42)

In [55]:
train_df_mini = train_df.sample(frac=0.1, random_state=42)
eval_df_mini = eval_df.sample(frac=0.1, random_state=42)
test_df_mini = test_df.sample(frac=0.1, random_state=42)

train_df_tiny = train_df.sample(frac=0.01, random_state=42)
eval_df_tiny = eval_df.sample(frac=0.01, random_state=42)
test_df_tiny = test_df.sample(frac=0.01, random_state=42)

In [8]:
train_df.to_csv('./train.csv', index=False)
train_df_mini.to_csv('./train_mini.csv', index=False)
train_df_tiny.to_csv('./train_tiny.csv', index=False)
eval_df.to_csv('./eval.csv', index=False)
eval_df_mini.to_csv('./eval_mini.csv', index=False)
eval_df_tiny.to_csv('./eval_tiny.csv', index=False)
test_df.to_csv('./test.csv', index=False)
test_df_mini.to_csv('./test_mini.csv', index=False)
test_df_tiny.to_csv('./test_tiny.csv', index=False)

### 데이터 추가 정제 (24.02.06)

In [11]:
total = pd.read_csv('./integrated_total/total.csv')
total.head()

,ko,en,source
0,통영 용남면 장평리 바닷가 해안도로 앞에 위치한 통영 늘푸른 펜션에 오신 것을 환영...,Welcome to Tongyeong's Evergreen Pension locat...,111
1,허브아일랜드의 불빛동화 축제는 국내 최고 겨울축제 중 하나로 꼽힌다.,Herb Island's Light Fairy Tale Festival is con...,111
2,봉비산 자락에 위치하고 있는 도심 속의 전통 사찰이다.,It is a traditional temple in the city center ...,111
3,수룡동 마을안길로 들어가면 쉽게 오를 수 있다.,It can be easily climbed by entering the insid...,111
4,"토스 측은 ""가족과 함께 송편을 나눠먹듯, 송편지원금 또한 가족, 친척, 친구, 오...","Toss explained the meaning of the event, sayin...",111


In [12]:
from transformers import AutoTokenizer

plm_name = 'beomi/open-llama-2-ko-7b'
tokenizer = AutoTokenizer.from_pretrained(plm_name)
tokenizer.pad_token = "</s>"
tokenizer.pad_token_id = 2
tokenizer.eos_token = "<|endoftext|>"
tokenizer.eos_token_id = 46332
tokenizer.add_eos_token = True
tokenizer.padding_side = 'right'
tokenizer.model_max_length = 768

In [18]:
from tqdm import tqdm

drop_rows = []
for idx, row in tqdm(total.iterrows(), total=len(total)):
    text = f"### English: {row['en']}\n### 한국어: {row['ko']}"
    outputs = tokenizer.encode_plus(
        text,
        padding=False,
        truncation=True,
        max_length=768,
        return_tensors='pt',
        return_attention_mask=False,
        return_length=False
    )
    
    input_len = len(outputs.input_ids.squeeze())
    if input_len > 700:
        drop_rows.append(idx)

print(len(drop_rows))

100%|██████████| 10416509/10416509 [34:08<00:00, 5085.48it/s]

5973


In [19]:
total_drop_long = total.drop(drop_rows)
total_drop_long.head()

,ko,en,source
0,통영 용남면 장평리 바닷가 해안도로 앞에 위치한 통영 늘푸른 펜션에 오신 것을 환영...,Welcome to Tongyeong's Evergreen Pension locat...,111
1,허브아일랜드의 불빛동화 축제는 국내 최고 겨울축제 중 하나로 꼽힌다.,Herb Island's Light Fairy Tale Festival is con...,111
2,봉비산 자락에 위치하고 있는 도심 속의 전통 사찰이다.,It is a traditional temple in the city center ...,111
3,수룡동 마을안길로 들어가면 쉽게 오를 수 있다.,It can be easily climbed by entering the insid...,111
4,"토스 측은 ""가족과 함께 송편을 나눠먹듯, 송편지원금 또한 가족, 친척, 친구, 오...","Toss explained the meaning of the event, sayin...",111


In [20]:
len(total_drop_long)

10410536

In [21]:
total_drop_long.to_csv('./integrated_total/total_drop_long.csv', index=False)